In [ ]:
import pandas as pd

# 1) 원본 CSV 읽기
# 파일 이름/경로 네 환경에 맞게 수정
df = pd.read_csv("recall.csv", encoding="utf-8")

# car_name_full 결측은 빈 문자열로
df["car_name_full"] = df["car_name_full"].fillna("")

# 2) car_name_full을 ',' 기준으로 나누고 explode로 행 늘리기
df_expanded = df.assign(
    model_name = df["car_name_full"].str.split(",")
).explode("model_name")

# 공백 제거
df_expanded["model_name"] = df_expanded["model_name"].astype(str).str.strip()
# model_name이 빈 문자열이면 버리기
df_expanded = df_expanded[df_expanded["model_name"] != ""]

# 3) 생산기간을 start / end 로 나누기
# 예: '2018-12-04 ~ 2020-10-24'
prod = df_expanded["생산기간"].fillna("")

# '~' 기준으로 앞/뒤 나누기
period_split = prod.str.split("~", n=1, expand=True)
df_expanded["생산_start_str"] = period_split[0].str.strip()
df_expanded["생산_end_str"]   = period_split[1].fillna("").str.strip()

# 문자열 → 날짜 (형식 안 맞으면 NaT로)
df_expanded["생산_start"] = pd.to_datetime(
    df_expanded["생산_start_str"],
    format="%Y-%m-%d",
    errors="coerce"
)
df_expanded["생산_end"] = pd.to_datetime(
    df_expanded["생산_end_str"].replace("", pd.NA),
    format="%Y-%m-%d",
    errors="coerce"
)

# 4) 대상수량을 숫자로 정리하고 싶으면 (선택)
if "대상수량" in df_expanded.columns:
    df_expanded["대상수량_int"] = (
        df_expanded["대상수량"]
        .astype(str)
        .str.replace(",", "", regex=False)
        .str.strip()
    )
    df_expanded["대상수량_int"] = pd.to_numeric(
        df_expanded["대상수량_int"],
        errors="coerce"
    )

# 5) 필요 없는 중간 컬럼 제거
df_expanded = df_expanded.drop(columns=["생산_start_str", "생산_end_str"])

# 6) 새 CSV로 저장
df_expanded.to_csv("recall_info_expanded.csv", index=False, encoding="utf-8-sig")

print("완료. 행 개수:", len(df), "→", len(df_expanded))


In [ ]:
%%sql
